In [14]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [17]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
import okada_wrapper

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [18]:
import celeri
celeri = reload(celeri)

In [19]:
# RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
# command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [20]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)

- Dip-slip rate constraint on Nevada02abb: rate = -2.40 (mm/yr), 1-sigma uncertainty = +/-1.20 (mm/yr)
- Dip-slip rate constraint on asdfbb: rate = -3.00 (mm/yr), 1-sigma uncertainty = +/-1.50 (mm/yr)
- Dip-slip rate constraint on cfma_newport_inglewood_complete_trace_part4_s: rate = 1.20 (mm/yr), 1-sigma uncertainty = +/-0.60 (mm/yr)
- Dip-slip rate constraint on cfma_raymond_complete_trace_part4_sa: rate = 1.50 (mm/yr), 1-sigma uncertainty = +/-0.80 (mm/yr)
- Dip-slip rate constraint on cfma_santa_susana_complete_trace_part4_s: rate = 6.00 (mm/yr), 1-sigma uncertainty = +/-3.90 (mm/yr)
- Dip-slip rate constraint on cfma_white_wolf_long_complete_trace_part6_s: rate = 2.30 (mm/yr), 1-sigma uncertainty = +/-4.40 (mm/yr)
- Dip-slip rate constraint on deepspringsvalley1b: rate = -1.00 (mm/yr), 1-sigma uncertainty = +/-0.80 (mm/yr)
- Dip-slip rate constraint on tahoeaabaa: rate = -0.80 (mm/yr), 1-sigma uncertainty = +/-0.50 (mm/yr)


In [9]:
# TODO: add tqdm to environment and install for celeri for progress monitoring
# TODO: rename G
# TODO: assemble G
# TODO: push text from above to log file.
# def get_segment_operator_okada(segment, stations, command):
"""
Calculates the elastic displacement partial derivatives based on the Okada
formulation, using the source and receiver geometries defined in
dicitonaries segment and stations. Before calculating the partials for
each segment, a local oblique Mercator project is done.
"""

n_stations = len(station)
n_segments = len(segment)
G = np.zeros((3 * n_stations, 3 * n_segments))
v1 = np.zeros(n_segments)
v2 = np.zeros(n_segments)
v3 = np.zeros(n_segments)

# Loop through each segment and calculate displacements
for i in range(n_segments):
# for i in range(1):
    print(i)
    u_east, u_north, u_up = celeri.get_okada_displacements(segment.lon1[i], segment.lat1[i], segment.lon2[i], segment.lat2[i], segment.locking_depth[i], segment.burial_depth[i], segment.dip[i], command.material_lambda, command.material_mu, 1, 0, 0, station.lon, station.lat)

#     % Displacements due to unit slip components
#     [ves vns vus...
#      ved vnd vud...
#      vet vnt vut] = okada_partials(ofx, ofy, strike, f.lDep, deg_to_rad(f.dip), L, W, 1, 1, 1, s.fpx, s.fpy, command.poissons_ratio);
     
#     v1{i} = reshape([ves vns vus]', 3*nSta, 1)
#     v2{i} = reshape(sign(90 - f.dip).*[ved vnd vud]', 3*nSta, 1)
#     v3{i} = reshape((f.dip - 90 == 0).*[vet vnt vut]', 3*nSta, 1) 
#     v1{i} = xyz2enumat((v1{i}), -f.strike + 90)
#     v2{i} = xyz2enumat((v2{i}), -f.strike + 90)
#     v3{i} = xyz2enumat((v3{i}), -f.strike + 90)

# Place cell arrays into the partials matrix
# G[:, 0::3] = cell2mat(v1)
# G[:, 1::3] = cell2mat(v2)
# G[:, 2::3] = cell2mat(v3) 


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
# TODO: Locking depths are currently meters rather than KM in inputfiles!!!
# TODO: Is there another XYZ to ENU conversion needed?

i = 0
segment.locking_depth.values[i] *= celeri.KM2M
segment.burial_depth.values[i] *= celeri.KM2M

# Project coordinates to flat space using a local oblique Mercator projection
projection = celeri.get_segment_oblique_projection(segment.lon1[i], segment.lat1[i], segment.lon2[i], segment.lat2[i], True)
station_x, station_y = projection(station.lon, station.lat)
segment_x1, segment_y1 = projection(segment.lon1[i], segment.lat1[i])
segment_x2, segment_y2 = projection(segment.lon2[i], segment.lat2[i])

# Calculate geometric fault parameters
segment_strike = np.arctan2(segment_y2 - segment_y1, segment_x2 - segment_x1) # radians
segment_length = np.sqrt((segment_y2 - segment_y1)**2.0 + (segment_x2 - segment_x1)**2.0)
segment_up_dip_width = (segment.locking_depth[i] - segment.burial_depth[i]) / np.sin(np.deg2rad(segment.dip[i]))

# Translate stations and segment so that segment mid-point is at the origin
segment_x_mid = (segment_x1 + segment_x2) / 2.0
segment_y_mid = (segment_y1 + segment_y2) / 2.0
station_x -= segment_x_mid
station_y -= segment_y_mid
segment_x1 -= segment_x_mid
segment_x2 -= segment_x_mid
segment_y1 -= segment_y_mid
segment_y2 -= segment_y_mid

# Unrotate coordinates to eliminate strike, segment will lie along y = 0
# np.einsum guidance from: https://stackoverflow.com/questions/26289972/use-numpy-to-multiply-a-matrix-across-an-array-of-points
rotation_matrix = np.array([[np.cos(segment_strike), -np.sin(segment_strike)], 
                            [np.sin(segment_strike), np.cos(segment_strike)]])
un_rotation_matrix = np.array([[np.cos(-segment_strike), -np.sin(-segment_strike)], 
                               [np.sin(-segment_strike), np.cos(-segment_strike)]])
station_x_rotated, station_y_rotated = np.hsplit(np.einsum("ij,kj->ik", np.dstack((station_x, station_y))[0], un_rotation_matrix), 2)
segment_x1_rotated, segment_y1_rotated = un_rotation_matrix.dot([segment_x1, segment_y1])
segment_x2_rotated, segment_y2_rotated = un_rotation_matrix.dot([segment_x2, segment_y2])

# Elastic displacements due to fault slip from Okada 1985
alpha = (command.material_lambda + command.material_mu) / (command.material_lambda + 2 * command.material_mu)
u_x = np.zeros_like(station_x)
u_y = np.zeros_like(station_x)
u_z = np.zeros_like(station_x)
for j in range(len(station)):
    _, u, _ = okada_wrapper.dc3dwrapper(alpha, # (lambda + mu) / (lambda + 2 * mu)
                          [station_x_rotated[j], station_y_rotated[j], 0], # (meters) observation point
                          -segment.locking_depth[i], # (meters) depth of the fault origin
                          segment.dip[i], # (degrees) the dip-angle of the rectangular dislocation surface
                          [-segment_length / 2, segment_length / 2], # (meters) the along-strike range of the surface (al1,al2 in the original)
                          [0, segment_up_dip_width], # (meters) along-dip range of the surface (aw1, aw2 in the original)
                          [1.0, 0.0, 0.0]) # (meters) strike-slip, dip-slip, tensile-slip
    u_x[j] = u[0]
    u_y[j] = u[1]
    u_z[j] = u[2]


# Un-rotate displacement to account for projected fault strike
u_x_un_rotated, u_y_un_rotated = np.hsplit(np.einsum("ij,kj->ik", np.dstack((u_x, u_y))[0], rotation_matrix), 2)



# plt.figure()
# plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-r")
# plt.plot(station.lon, station.lat, '.b', markersize=1)
# plt.xlim([235, 255])
# plt.ylim([30, 50])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Positions: longitude and latitude")
# plt.show()

# plt.figure()
# plt.plot([segment_x1, segment_x2], [segment_y1, segment_y2], "-r")
# plt.plot(station_x, station_y, '.b', markersize=1)
# plt.xlim([-1e6, 1e6])
# plt.ylim([-1e6, 1e6])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Positions: projected and translated")
# plt.show()

# plt.figure()
# plt.plot([segment_x1_rotated, segment_x2_rotated], [segment_y1_rotated, segment_y2_rotated], "-r")
# plt.plot(station_x_rotated, station_y_rotated, '.b', markersize=1)
# plt.xlim([-1e6, 1e6])
# plt.ylim([-1e6, 1e6])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Positions: projected, translated, and rotated")
# plt.show()

# plt.figure()
# plt.plot([segment_x1_rotated, segment_x2_rotated], [segment_y1_rotated, segment_y2_rotated], "-r")
# # plt.plot(station_x_rotated, station_y_rotated, '.b', markersize=1)
# plt.quiver(station_x_rotated, station_y_rotated, u_x, u_y, scale=1e-1, scale_units='inches')
# plt.xlim([-1e6, 1e6])
# plt.ylim([-1e6, 1e6])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Displacements: projected, translated, and rotated")
# plt.show()

# plt.figure()
# plt.plot([segment_x1, segment_x2], [segment_y1, segment_y2], "-r")
# # plt.plot(station_x, station_y, '.b', markersize=1)
# plt.quiver(station_x, station_y, u_x_un_rotated, u_y_un_rotated, scale=1e-1, scale_units='inches')
# plt.xlim([-1e6, 1e6])
# plt.ylim([-1e6, 1e6])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.title("Displacements: projected and translated")
# plt.show()

plt.figure()
plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-r")
# plt.plot(station_x, station_y, '.b', markersize=1)
plt.quiver(station.lon, station.lat, u_x_un_rotated, u_y_un_rotated, scale=1e-1, scale_units='inches')
plt.xlim([235, 255])
plt.ylim([30, 50])
plt.gca().set_aspect('equal', adjustable='box')
plt.title("Displacements: longitude and latitude")
plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
i = 0
u_east, u_north, u_up = celeri.get_okada_displacements(segment.lon1.values[i], segment.lat1[i], segment.lon2[i], segment.lat2[i], segment.locking_depth[i], segment.burial_depth[i], segment.dip[i], command.material_lambda, command.material_mu, 1, 0, 0, station.lon, station.lat)
plt.figure()

plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-r")
plt.quiver(station.lon, station.lat, u_x_un_rotated, u_y_un_rotated, scale=1e-1, scale_units='inches')
plt.xlim([235, 255])
plt.ylim([30, 50])
plt.gca().set_aspect('equal', adjustable='box')
plt.title("Displacements: longitude and latitude")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
celeri.RUN_NAME

'210826120510/'